In [1]:
import os
import re
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import spacy
import tensorflow as tf
import tensorflow_hub as hub
import swifter

from news_utils.clean.english import clean

/home/group7/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from itertools import islice

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [3]:
pd.options.display.max_colwidth = 1000

In [4]:
PATH_CL_TR = Path('~/data/ynacc_proc/replicate/split/train.csv')
PATH_CL_VA = Path('~/data/ynacc_proc/replicate/split/val.csv')

In [4]:
df = pd.read_csv(PATH_CL_TR)

In [5]:
df[['article_text', 'text']][:1]

,article_text,text
0,"A Disneyland Paris employee was found dead inside the park's Phantom Manor ride on Saturday, Le Parisien reports. Investigators believe he was a maintenance officer who was fixing a faulty light fixture and accidentally electrocuted.\n\nThough he's yet to be officially identified, a Disneyland union representative told Le Parisien the worker was a 45-year-old man who ""always had a smile on his face"" and was ""very popular with his colleagues."" The park remained open after his body was found, but the ride itself — described as a ""fun-filled mystery"" filled with ""ghoulish ghosts and spirits"" on the park's website — remains closed.\n\nAdvertisement - Continue Reading Below\n\nAccording to the BBC, this is not the first employee death the park has seen: In 2010, a maintenance worker was cleaning the Small World ride, fell into the water, and was trapped under a boat.\n\nFollow Tess on Twitter.",Yes..because too many houses in EU look like the original Disney Hunted House so it didn't look scary enough. Bates Motel looks more American and that notion alone scares everyone.


In [6]:
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")
nlp = spacy.load('en_core_web_lg')

NameError: name 'hub' is not defined

In [7]:
def get_embeddings(session_, input_tensor_, text, encoding_tensor):
    return session_.run(
      encoding_tensor, feed_dict={input_tensor_: text})

In [8]:
def process_row(x):
    data = []

    atextdoc = nlp(clean(str(x['text_article']), lower=False))
    ctextdoc = nlp(clean(str(x['text']), lower=False))

    articles = []
    comments = []

    for s in ctextdoc.sents:
        comments.append(str(s))

    comments = [c for c in comments if len(c) > 20]

#     if len(comments) < 2:
#         return clean(str(x['text']))

    comments = window(comments, 2)
    comments = [' '.join(w) for w in comments]

    for s in atextdoc.sents:
        articles.append(str(s))
    articles = [a for a in articles if len(
        a) > 20 and not 'Advertisement - Continue Reading Below'.lower() in a.lower()]
    articles = window(articles, 2)
    articles = [' '.join(w) for w in articles]

    return [articles, comments]


def process(articles_bla, comments_bla):
    comments_fin = articles_bla.merge(
        comments_bla, how="left", on="url", suffixes=('_article', ''))
    print(articles_bla.shape, comments_bla.shape)
    comments_raw = comments_fin['text'].tolist()
    datas = comments_fin.swifter.apply(process_row)

#     remeber the number of sentence pairs for each comment
    remember_com = []
    remember_art = []
    for d in datas:
        remember_com.append(len(d[1]))
        remember_art.append(len(d[0]))

#     flatten
    articles_flat = sum([x[0] for x in datas], [])
    comments_flat = sum([x[1] for x in datas], [])

    similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
    similarity_message_encodings = embed(similarity_input_placeholder)
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
    #             for [articles, comments] in data:
        comment_embeddings = get_embeddings(
            session, similarity_input_placeholder, comments_flat, similarity_message_encodings)
        article_embeddings = get_embeddings(
            session, similarity_input_placeholder, articles_flat, similarity_message_encodings)

        #  now re-construct the old arrays
        
        running_idx = 0
        res_comments = []
        for x in remember_com:
            temp_res = []
            for i in range(x):
                temp_res.append(comment_embeddings[running_idx + i])
            running_idx = x
            res_comments.append(temp_res)
        running_idx = 0
        res_articles = []
        for x in remember_art:
            temp_res = []
            for i in range(x):
                temp_res.append(article_embeddings[running_idx + i])
            running_idx = x
            res_articles.append(temp_res)

        assert len(res_comments) == len(res_articles)

        for com_idx, [article_emb, comments_emb] in enumerate(zip(res_articles, res_comments)):
            #             print(article_emb)
            #             print(comments_emb)
            if len(comments_emb) == 0:
                continue
            corr = np.inner(article_emb, comments_emb)
            values = np.amax(corr, axis=0)
            idx = np.argmax(corr, axis=0)
    #         new_text = x['text']
            for i, [value, article_idx] in enumerate(zip(values, idx)):
                if value > 0.3:
                    #                     new_text = new_text.replace(comments[i], 'xx_related_start' + comments[i] + 'xx_related_end')
                    print(comments_raw[com_idx + i])
    #                     print(comments[i].strip())
    #                     print()
    #                     print(articles[article_idx].strip())
    #                     print(value)
    #                     print()
    #                     print()
#         return clean(new_text)
#         print(new_text)

#         comments_fin['text_proc']

In [5]:
# Do it for the classification
PATH_ARTICLES = Path('/mnt/data/group07/johannes/ynacc_proc/articles/articles_fixed_5.csv')
PATH_CL_TR = Path('~/data/ynacc_proc/replicate/split/train.csv')
PATH_CL_VA = Path('~/data/ynacc_proc/replicate/split/val.csv')

In [6]:
df_art = pd.read_csv(PATH_ARTICLES)
df_cl_tr = pd.read_csv(PATH_CL_TR)
df_cl_va = pd.read_csv(PATH_CL_VA)

In [11]:
df_cl_out_tr = process(df_art[:2], df_cl_tr)

(2, 6) (7910, 36)


AttributeError: 'DataFrame' object has no attribute 'swifter'

In [16]:
df_cl_tr

,Unnamed: 0,Unnamed: 0.1,url,article_text,title,publish_date,commentid,clcontroversial,clmean,clinformative,...,sd_agreement,sd_type,sentiment,tone,commentagreement,topic,intendedaudience,persuasiveness,text_proc,cltopic
0,0,0,http://www.cosmopolitan.com/lifestyle/news/a56215/disneyland-paris-haunted-mansion-death/,"A Disneyland Paris employee was found dead inside the park's Phantom Manor ride on Saturday, Le Parisien reports. Investigators believe he was a maintenance officer who was fixing a faulty light fixture and accidentally electrocuted.\n\nThough he's yet to be officially identified, a Disneyland union representative told Le Parisien the worker was a 45-year-old man who ""always had a smile on his face"" and was ""very popular with his colleagues."" The park remained open after his body was found, but the ride itself — described as a ""fun-filled mystery"" filled with ""ghoulish ghosts and spirits"" on the park's website — remains closed.\n\nAdvertisement - Continue Reading Below\n\nAccording to the BBC, this is not the first employee death the park has seen: In 2010, a maintenance worker was cleaning the Small World ride, fell into the water, and was trapped under a boat.\n\nFollow Tess on Twitter.",Disneyland Worker Found Dead in Haunted Mansion,2016-04-04 03:03:48,00002n000000000000000000000000-1c30b878-b717-4e9a-9872-2ce2906ce783,0,0,1,...,NaN,Positive/respectful,neutral,Informative,NaN,Off-topic with article,Reply to a specific commenter,Not persuasive,Yes..because too many houses in EU look like the original Disney Hunted House so it didn't look scary enough. Bates Motel looks more American and that notion alone scares everyone.,0
1,1,1,http://www.cosmopolitan.com/lifestyle/news/a56215/disneyland-paris-haunted-mansion-death/,"A Disneyland Paris employee was found dead inside the park's Phantom Manor ride on Saturday, Le Parisien reports. Investigators believe he was a maintenance officer who was fixing a faulty light fixture and accidentally electrocuted.\n\nThough he's yet to be officially identified, a Disneyland union representative told Le Parisien the worker was a 45-year-old man who ""always had a smile on his face"" and was ""very popular with his colleagues."" The park remained open after his body was found, but the ride itself — described as a ""fun-filled mystery"" filled with ""ghoulish ghosts and spirits"" on the park's website — remains closed.\n\nAdvertisement - Continue Reading Below\n\nAccording to the BBC, this is not the first employee death the park has seen: In 2010, a maintenance worker was cleaning the Small World ride, fell into the water, and was trapped under a boat.\n\nFollow Tess on Twitter.",Disneyland Worker Found Dead in Haunted Mansion,2016-04-04 03:03:48,00003n000000000000000000000000-ed2ae6d0-32ac-471a-b8b2-a718607ee376,0,0,0,...,NaN,Positive/respectful,negative,NaN,Disagreement with commenter,Off-topic with article,Reply to a specific commenter,Not persuasive,"These things happen , Every job has its dangers.",1
2,2,2,http://www.cosmopolitan.com/lifestyle/news/a56215/disneyland-paris-haunted-mansion-death/,"A Disneyland Paris employee was found dead inside the park's Phantom Manor ride on Saturday, Le Parisien reports. Investigators believe he was a maintenance officer who was fixing a faulty light fixture and accidentally electrocuted.\n\nThough he's yet to be officially identified, a Disneyland union representative told Le Parisien the worker was a 45-year-old man who ""always had a smile on his face"" and was ""very popular with his colleagues."" The park remained open after his body was found, but the ride itself — described as a ""fun-filled mystery"" filled with ""ghoulish ghosts and spirits"" on the park's website — remains closed.\n\nAdvertisement - Continue Reading Below\n\nAccording to the BBC, this is not the first employee death the park has seen: In 2010, a maintenance worker was cleaning the Small World ride, fell into the water, and was trapped under a boat.\n\nFol

In [18]:
df_cl_tr[['text', 'cltopic', 'parentid']][:100]

,text,cltopic,parentid
0,Yes..because too many houses in EU look like the original Disney Hunted House so it didn't look scary enough. Bates Motel looks more American and that notion alone scares everyone.,0,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b4249
1,"These things happen , Every job has its dangers.",1,1459879464596-a3771c05-fd2e-4f44-a26a-23baec3b4249
2,"Sad to hear such a bad thing. Very dangerous job working on electricity. One questions though, why did they use a picture the Bates house from Psycho, on a Disney story? Or is that what the Paris Haunted Mansion/Phantom Manor looks like?",1,NaN
3,THANK YOU!!!!!!!!!!!!!!!! So annoying.. journalism today is questionable... @USERNAME,1,1459864307079-aeebbc73-f562-4540-8576-cdcebe13f099
4,lol i thought it's my computer,1,1459864307079-aeebbc73-f562-4540-8576-cdcebe13f099
5,It's on the Cosmopolitan article you dolt.,1,1459864307079-aeebbc73-f562-4540-8576-cdcebe13f099
6,You have to click on the link to go to Cosmo. Yahoo has been terrible about this lately.,1,1459864307079-aeebbc73-f562-4540-8576-cdcebe13f099
7,Is it just my computer or is there not a single picture of her post-baby belly on this article? I didn't even see one on her Twitter account. Good grief.,1,NaN
8,why is she sick --American laws are sick --16 is not a kid,0,1460818837098-c6d3ff5e-f271-4f00-8fc5-935aa5429e72
9,"Still, the woman is the legal adult and is a moron for not taking better care of herself. I agree that the whole ""victim"" thing here is vague, though. For a 16 year old boy, banging the hot teacher is not abuse. He's still doing high fives over it.",1,1460793264228-79890a84-148f-461e-b662-4a928c4b4c7f


In [168]:
df_cl_tr[df_cl_tr['text'].str.contains('THANK YOU')]['text']

3                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         THANK YOU!!!!!!!!!!!!!!!! So annoying.. journalism today is questionable... @U

In [169]:
df_art

,Unnamed: 0,Unnamed: 0.1,url,text,title,publish_date
0,0,1,http://www.cosmopolitan.com/lifestyle/news/a56215/disneyland-paris-haunted-mansion-death/,"A Disneyland Paris employee was found dead inside the park's Phantom Manor ride on Saturday, Le Parisien reports. Investigators believe he was a maintenance officer who was fixing a faulty light fixture and accidentally electrocuted.\n\nThough he's yet to be officially identified, a Disneyland union representative told Le Parisien the worker was a 45-year-old man who ""always had a smile on his face"" and was ""very popular with his colleagues."" The park remained open after his body was found, but the ride itself — described as a ""fun-filled mystery"" filled with ""ghoulish ghosts and spirits"" on the park's website — remains closed.\n\nAdvertisement - Continue Reading Below\n\nAccording to the BBC, this is not the first employee death the park has seen: In 2010, a maintenance worker was cleaning the Small World ride, fell into the water, and was trapped under a boat.\n\nFollow Tess on Twitter.",Disneyland Worker Found Dead in Haunted Mansion,2016-04-04 03:03:48
1,1,2,http://www.cosmopolitan.com/health-fitness/news/a56117/chontel-duncan-post-baby-stomach/,"Having documented her baby bump in mirror selfies throughout her whole pregnancy, it's no surprise fitness model Chontel Duncan shared a full-length selfie just five days after welcoming her baby boy. And yeah, she looks as freakin' amazing as you would expect.\n\nAdvertisement - Continue Reading Below\n\n""My body is tender, it feels like someone did 12 rounds in the boxing ring using my core as the target,"" Duncan, who delivered her son Jeremiah by C-section, wrote on Instagram. ""So much swelling is yet to go down and so much healing is yet to happen. ... It's super crazy how different my body feels, I can't believe my bump is gone and I no longer have my baby hiding inside of me.""\n\nShe also acknowledged that she won't be hitting the gym for a while, and she wonders how her body will change ""through purely just recovery/rest and nutrition."" She's welcoming the timeout, though, noting, ""I'm excited because it gives me more time with Miah!""\n\nSuch a proud mama.\n\nMy Mornings are...",See Fitness Model Chontel Duncan's Post-Baby Belly 5 Days After Giving Birth,2016-04-01 03:17:00
2,2,3,http://www.insideedition.com/headlines/15888-english-teacher-charged-with-sexual-assault-of-a-student-youre-my-baby-boo,"Headlines Teacher Allegedly Had Sex With Student On Night of Her Husband's Bachelor Party Playing Teacher Allegedly Had Sex With Student On Night of Her Husband's Bachelor Party\n\n\n\n\n\n\n\nA Wisconsin high school teacher is facing charges of sexual assault following an alleged sexual relationship with a student, including on the night of her husband's bachelor's party.\n\nRead: Daycare Center Shuttered After 3-Month-Old Died On Her Mother's First Day Back at Work\n\n\n\nSara Domres, a 29-year-old former teacher at the New Berlin West High School, was charged last week for sexual assault of a student by school staff following an alleged relationship with a 16-year-old lasting approximately ten months.\n\nAccording to the criminal complaint, the sophomore was in Domres' English class, where they became fast friends, and texted each other frequently, the police report stated.\n\nDuring their relationship, lasting between April 2015 and January 2016, they reportedly exchanged over ...",Teacher Allegedly Had Sex With Student On Night of Her Husband's Bachelor Party,NaN
3,3,4,http://www.ozy.com/good-sht/the-dirtiest-soccer-player-in-the-known-universe/68373,"Because never does a man stand so tall as when he stoops to stomp on the hand of a man in need.\n\nThere’s a twisted delight to be had in watching Real Madrid’s oft-injured central defender sensation, Képler Laveran Lima Ferreira, or Pepe, do what he does best. During the 2014 FIFA World Cup, playing for Portugal against Germany, Pepe walked away, leaving Thomas Mü

In [170]:
df_art[df_art['text'].duplicated()]

,Unnamed: 0,Unnamed: 0.1,url,text,title,publish_date
